In [1]:
#!/usr/bin/env python
# coding: utf-8

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1691628408595_0004,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import boto3
import os
import sys
import pytz
import numpy as np
import datetime
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, lpad, translate
from datetime import datetime 
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
agora=datetime.now(pytz.timezone('America/Sao_Paulo'))
dthproc=agora.strftime("%Y%m%d%H%M%S")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark = SparkSession.builder.appName('trusted_orders').getOrCreate()
sqlContext=SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
_DIA_ = '20230804'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
bucket_refined = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/004_refined'.format(dia=_DIA_)
bucket_trusted = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/003_trusted'.format(dia=_DIA_)
bucket_raw = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/002_raw'.format(dia=_DIA_)
bucket_control = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/005_control'.format(dia=_DIA_)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
output_refined = 't_indicadores_segmentos_clientes'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
path_lake = os.path.join('s3://',bucket_trusted,'trusted_application_train_20230805')
application_train_00=sqlContext.read.format("parquet").load(path_lake)
application_train_00.registerTempTable("application_train_00")
application_train_00.cache()
application_train_00.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

307511

In [11]:
application_train_00.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ref: integer (nullable = true)
 |-- ts_proc: long (nullable = true)
 |-- SK_ID_CURR: integer (nullable = true)
 |-- TARGET: integer (nullable = true)
 |-- NAME_CONTRACT_TYPE: string (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- AMT_CREDIT: double (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- AMT_GOODS_PRICE: double (nullable = true)
 |-- NAME_TYPE_SUITE: string (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- REGION_POPULATION_RELATIVE: double (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- DAYS_REGISTRATION: double (nullabl

In [18]:
indicadores_00 = spark.sql(     
    """
        SELECT
            TARGET,
            AMT_CREDIT,
            OCCUPATION_TYPE,
            ORGANIZATION_TYPE,
            NAME_EDUCATION_TYPE,
            NAME_FAMILY_STATUS,
            NAME_INCOME_TYPE,
            NAME_TYPE_SUITE,
            CASE
                WHEN TARGET = 0 THEN "Sem atraso"
            ELSE "Inadimplente"
            END AS TARGET_traduzido
        FROM
            application_train_00
            
    """)
indicadores_00.registerTempTable("indicadores_00")
indicadores_00.cache()
indicadores_00.count() 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

307511

In [19]:
indicadores_00.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+---------------+--------------------+--------------------+--------------------+--------------------+---------------+----------------+
|TARGET|AMT_CREDIT|OCCUPATION_TYPE|   ORGANIZATION_TYPE| NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|    NAME_INCOME_TYPE|NAME_TYPE_SUITE|TARGET_traduzido|
+------+----------+---------------+--------------------+--------------------+--------------------+--------------------+---------------+----------------+
|     1|  406597.5|       Laborers|Business Entity T...|Secondary / secon...|Single / not married|             Working|  Unaccompanied|    Inadimplente|
|     0| 1293502.5|     Core staff|              School|    Higher education|             Married|       State servant|         Family|      Sem atraso|
|     0|  135000.0|       Laborers|          Government|Secondary / secon...|Single / not married|             Working|  Unaccompanied|      Sem atraso|
|     0|  312682.5|       Laborers|Business Entity T...|Secondary / secon...|     

In [20]:
# +------+----------+---------------+--------------------+--------------------+--------------------+--------------------+---------------+----------------+
# |TARGET|AMT_CREDIT|OCCUPATION_TYPE|   ORGANIZATION_TYPE| NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|    NAME_INCOME_TYPE|NAME_TYPE_SUITE|TARGET_traduzido|
# +------+----------+---------------+--------------------+--------------------+--------------------+--------------------+---------------+----------------+
# |     1|  406597.5|       Laborers|Business Entity T...|Secondary / secon...|Single / not married|             Working|  Unaccompanied|    Inadimplente|
# |     0| 1293502.5|     Core staff|              School|    Higher education|             Married|       State servant|         Family|      Sem atraso|
# |     0|  135000.0|       Laborers|          Government|Secondary / secon...|Single / not married|             Working|  Unaccompanied|      Sem atraso|
# |     0|  312682.5|       Laborers|Business Entity T...|Secondary / secon...|      Civil marriage|             Working|  Unaccompanied|      Sem atraso|
# |     0|  513000.0|     Core staff|            Religion|Secondary / secon...|Single / not married|             Working|  Unaccompanied|      Sem atraso|
# |     0|  490495.5|       Laborers|               Other|Secondary / secon...|             Married|       State servant|Spouse, partner|      Sem atraso|
# |     0| 1560726.0|    Accountants|Business Entity T...|    Higher education|             Married|Commercial associate|  Unaccompanied|      Sem atraso|
# |     0| 1530000.0|       Managers|               Other|    Higher education|             Married|       State servant|  Unaccompanied|      Sem atraso|
# |     0| 1019610.0|           null|                 XNA|Secondary / secon...|             Married|           Pensioner|       Children|      Sem atraso|
# |     0|  405000.0|       Laborers|         Electricity|Secondary / secon...|Single / not married|             Working|  Unaccompanied|      Sem atraso|
# |     0|  652500.0|     Core staff|            Medicine|    Higher education|             Married|             Working|  Unaccompanied|      Sem atraso|
# |     0|  148365.0|           null|                 XNA|Secondary / secon...|             Married|           Pensioner|       Children|      Sem atraso|
# |     0|   80865.0|       Laborers|Business Entity T...|Secondary / secon...|             Married|             Working|  Unaccompanied|      Sem atraso|
# |     0|  918468.0|        Drivers|       Self-employed|Secondary / secon...|             Married|             Working|  Unaccompanied|      Sem atraso|
# |     0|  773680.5|       Laborers|   Transport: type 2|Secondary / secon...|             Married|             Working|  Unaccompanied|      Sem atraso|
# |     0|  299772.0|       Laborers|Business Entity T...|Secondary / secon...|Single / not married|             Working|         Family|      Sem atraso|
# |     0|  509602.5|        Drivers|          Government|Secondary / secon...|             Married|             Working|  Unaccompanied|      Sem atraso|
# |     0|  270000.0|       Laborers|        Construction|Secondary / secon...|             Married|             Working|  Unaccompanied|      Sem atraso|
# |     0|  157500.0|       Laborers|             Housing|Secondary / secon...|               Widow|             Working|        Other_A|      Sem atraso|
# |     0|  544491.0|     Core staff|        Kindergarten|    Higher education|Single / not married|       State servant|  Unaccompanied|      Sem atraso|
# +------+----------+---------------+--------------------+--------------------+--------------------+--------------------+---------------+----------------+
# only showing top 20 rows

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
tabela = output_refined

path_athena = os.path.join('s3://',bucket_refined,tabela)
print('path_tabela:', path_athena)
indicadores_00.coalesce(1).write.\
format("parquet").mode("overwrite").option("compression", "gzip").option("overwriteSchema", "true").\
option("path", path_athena).saveAsTable(tabela)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

path_tabela: s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/004_refined/t_indicadores_segmentos_clientes